In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df=spark.read.format("delta") \
    . option("header", "true") \
    . option("inferSchema", "true") \
    . load("dbfs:/mnt/bronze/netflix_titles")

In [0]:
df.display()

In [0]:
df = df.fillna({"duration_minutes": 0, "duration_seasons" :1 })

In [0]:
df=df.withColumn("duration_minutes", col("duration_minutes").cast("int")) \
    .withColumn("duration_seasons", col("duration_seasons").cast("int"))



In [0]:
df=df.withColumn("ShortTitle", split(col("title"), ":")[0]).drop("title")
df.display()

In [0]:
df=df.withColumn("rating", split(col("rating"), "-")[0])
df.display()

In [0]:
from pyspark.sql.functions import col, when

df = df.withColumn(
    "typle_flag",
    when(col("type") == "Movie", 1)
    .when(col("type") == "TV Show", 2)
    .otherwise(0)
)
display(df)

In [0]:
from pyspark.sql.window import Window

In [0]:
#df=df.withColumn("Duration_ranking", dense_rank().over(Window.orderBy(col('duration_minutes').desc())))

In [0]:
df_vis=df.groupBy("type").agg(count("type").alias("count"))
display(df_vis)

Databricks visualization. Run in Databricks to view.

In [0]:

# Write the cleaned DataFrame to Delta format
df.write.format("delta") \
    .mode("overwrite") \
    .option("path", "dbfs:/mnt/silver/netflix_titles") \
    .save()